# 資料合併-短程

In [11]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/short'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/tokyo.csv',
    f'{base_path}/hongkong.csv',
    f'{base_path}/singapore.csv',
    f'{base_path}/bangkok.csv',
    f'{base_path}/seoul.csv'
]

# 合併檔案
merge_files(file_paths, output_path)

正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/tokyo.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/hongkong.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/singapore.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/bangkok.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/seoul.csv
合併完成，結果儲存至：/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv


# 資料合併-短程-商務艙

In [ ]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/short'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight_business.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/tokyo_business.csv',
    f'{base_path}/hongkong_business.csv',
    f'{base_path}/singapore_business.csv',
    f'{base_path}/bangkok_business.csv',
    f'{base_path}/seoul_business.csv'
]

# 合併檔案
merge_files(file_paths, output_path)

# 短程航班-經濟艙

In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 載入資料並刪除價格歷史為空的列
data = pd.read_csv('/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv')

# 統一日期格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# 計算對數變數
data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)

# 91 - '最低價格剩餘天數' 
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除包含 NaN 的數據
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

抵達機場代號
HKG    2293
NRT    1252
BKK     949
ICN     768
SIN     595
HND     587
GMP     121
Name: count, dtype: int64


In [3]:
# 定義假期範圍
holidays = {
    "台灣": [("2025-01-25", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "日本": [("2025-02-11", "2025-02-11"), ("2025-02-22", "2025-02-24"), ("2025-03-20", "2025-03-20")],
    "香港": [("2025-01-29", "2025-01-31")],
    "新加坡": [("2025-01-29", "2025-01-30")],
    "韓國": [("2025-01-28", "2025-01-30"), ("2025-03-01", "2025-03-03")]
}

# 抵達機場代號與地區的對應關係
airport_to_region = {
    "HKG": "香港",
    "NRT": "日本",
    "HND": "日本",
    "BKK": "泰國",  # 泰國沒有假期，不處理
    "ICN": "韓國",
    "GMP": "韓國",
    "SIN": "新加坡"
}

# 轉換日期類型
data["出發日期"] = pd.to_datetime(data["出發日期"])

# 定義一個函數來檢查是否為假期
def is_holiday(row):
    # 台灣的假期（不管去哪，出發地是台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1  # 假期
    # 其他地區的假期
    region = airport_to_region.get(row["抵達機場代號"])
    if not region or region not in holidays:
        return 0  # 沒有假期
    for start, end in holidays[region]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1  # 假期
    return 0  # 非假期

# 新增假期欄位
data["假期"] = data.apply(is_holiday, axis=1)

# 調整「假期」到「艙等」後面
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# 新增 "Cost of Living Index" 和 "GDP (PPP) per capita (in thousand USD)" 資料
additional_data = {
    "Region": ["香港", "日本", "新加坡", "韓國", "泰國"],
    "Cost of Living Index": [70.8, 46.1, 76.7, 60.1, 34.1],
    "GDP (PPP) per capita (in thousand USD)": [71.627, 51.399, 141.553, 60.046, 23.981],
}

# 將 Region 對應到現有的抵達機場代號的地區
region_df = pd.DataFrame(additional_data)

# 透過 `airport_to_region` 映射，新增這兩個欄位到原始資料
data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# 定義平日與假日的對應關係
weekend_days = ["週六", "週日"]

# 新增 是否為平日 欄位，假日為 1，平日為 0
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將 是否為平日 欄位移到 是否過夜 之後
columns = data.columns.tolist()
idx = columns.index("是否過夜") + 1  # 找到 是否過夜 的索引位置
columns.insert(idx, columns.pop(columns.index("是否為平日")))  # 插入到 是否過夜 之後

# 重新排列 DataFrame
data = data[columns]

# 匯出新的資料集
data.to_csv('/Users/yuchingchen/Documents/專題/cleaned_data/short_flight_final.csv', index=False)

# 短程航班-商務艙

In [29]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 載入資料並刪除價格歷史為空的列
data = pd.read_csv('/Users/yuchingchen/Documents/專題/cleaned_data/short_flight_business.csv')

# 統一日期格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# 計算對數變數
data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)

# 91 - '最低價格剩餘天數' 
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除包含 NaN 的數據
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

抵達機場代號
HKG    1719
BKK     747
NRT     735
ICN     488
HND     476
SIN     418
GMP      54
Name: count, dtype: int64


In [1]:
# 定義假期範圍
holidays = {
    "台灣": [("2025-01-25", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "日本": [("2025-02-11", "2025-02-11"), ("2025-02-22", "2025-02-24"), ("2025-03-20", "2025-03-20")],
    "香港": [("2025-01-29", "2025-01-31")],
    "新加坡": [("2025-01-29", "2025-01-30")],
    "韓國": [("2025-01-28", "2025-01-30"), ("2025-03-01", "2025-03-03")]
}

# 抵達機場代號與地區的對應關係
airport_to_region = {
    "HKG": "香港",
    "NRT": "日本",
    "HND": "日本",
    "BKK": "泰國",  # 泰國沒有假期，不處理
    "ICN": "韓國",
    "GMP": "韓國",
    "SIN": "新加坡"
}

# 轉換日期類型
data["出發日期"] = pd.to_datetime(data["出發日期"])

# 定義一個函數來檢查是否為假期
def is_holiday(row):
    # 台灣的假期（不管去哪，出發地是台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1  # 假期
    # 其他地區的假期
    region = airport_to_region.get(row["抵達機場代號"])
    if not region or region not in holidays:
        return 0  # 沒有假期
    for start, end in holidays[region]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1  # 假期
    return 0  # 非假期

# 新增假期欄位
data["假期"] = data.apply(is_holiday, axis=1)

# 調整「假期」到「艙等」後面
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# 新增 "Cost of Living Index" 和 "GDP (PPP) per capita (in thousand USD)" 資料
additional_data = {
    "Region": ["香港", "日本", "新加坡", "韓國", "泰國"],
    "Cost of Living Index": [70.8, 46.1, 76.7, 60.1, 34.1],
    "GDP (PPP) per capita (in thousand USD)": [71.627, 51.399, 141.553, 60.046, 23.981],
}

# 將 Region 對應到現有的抵達機場代號的地區
region_df = pd.DataFrame(additional_data)

# 透過 `airport_to_region` 映射，新增這兩個欄位到原始資料
data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# 定義平日與假日的對應關係
weekend_days = ["週六", "週日"]

# 新增 是否為平日 欄位，假日為 1，平日為 0
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將 是否為平日 欄位移到 是否過夜 之後
columns = data.columns.tolist()
idx = columns.index("是否過夜") + 1  # 找到 是否過夜 的索引位置
columns.insert(idx, columns.pop(columns.index("是否為平日")))  # 插入到 是否過夜 之後

# 重新排列 DataFrame
data = data[columns]

# 匯出新的資料集
data.to_csv('/Users/yuchingchen/Documents/專題/cleaned_data/short_flight_business_final.csv', index=False)

NameError: name 'pd' is not defined